# AI201 - Programming Assignment 1
### Implementing the A* Algorithm
*Instructor:	Pros Naval*<br>
*Submitted by:  Mike Allan Nillo*

#### A* Pseduo-code
1. Put the start node s on a list called OPEN and compute f(s).
2. If OPEN is empty, exit with failure; otherwise continue.
3. Remove from OPEN that node whose f value is smallest and put it on a list called CLOSED. Call this node n. (Resolve ties for minimal f values arbitrarily, but always in favor of any goal node.)
4. If n is a goal node, exit with the solution path obtained by tracing back the pointers; otherwise continue.
5. Expand node n, generating all of its successors. If there are no successors, go immediately to 2. For each successor ni, compute f(n).
6. Associate with the successors not already on either OPEN or CLOSED the ƒ values just computed. Put these nodes on OPEN and direct pointers from them back to n.
7. Associate with those successors that were already on OPEN or CLOSED the smaller of the ƒ values just computed and their previous f values. Put on OPEN those successors on CLOSED whose ƒ values were thus lowered, and redirect to n the pointers from all nodes whose ƒ values were lowered.
8. Go to 2.

*Comments:*
- duplicates are not retained; when nodes are rediscovered, the ancestor history is updated
- when a successor is already on OPEN or CLOSED, the algorithm modifies the pointers so that the nodes record the shorter of the two partial paths

In [1]:
# Import necessary libraries 
import numpy as np

In [2]:
# Defining the heuristic function for the A* algorithm to solve the 8-puzzle problem by evaluating the number of tiles in the wrong position
def heuristic(state, goal_state):
    h = 0
    for i in range(9):
        if state[i] != goal_state[i]:
            h += 1
    return h

# Defining the heuristic function for the A* algorithm to solve the 8-puzzle problem by evaluating the Manhattan distance
def heuristic_manhattan(state, goal_state):
    h = 0
    for i in range(9):
        if state[i] != goal_state[i]:
            h += abs(i // 3 - goal_state.index(state[i]) // 3) + abs(i % 3 - goal_state.index(state[i]) % 3)
    return h

# Defining the heuristic function for the A* algorithm to solve the 8-puzzle problem by evaluating the Nilsson's sequence score

def heuristic_nilsson(state, goal_state):
    p_n = 0
    s_n = 0
    h = 0

    # Define the correct sequence of tiles around the edges
    correct_sequence = [1, 2, 3, 8, 0, 4, 7, 6, 5]

    # Create a list of the current sequence of tiles around the edges
    current_sequence = [state[correct_sequence.index(i)] for i in range(1, 9)]

    # Calculate S(n)
    for i in range(8):  # We only go up to 8 because the last tile wraps around to the first
        if current_sequence[i] != correct_sequence[i] or current_sequence[(i+1)%8] != correct_sequence[(i+1)%8]:
            s_n += 2

    # Calculate P(n)
    for i in range(9):
        if state[i] != goal_state[i]:
            p_n += abs(i // 3 - goal_state.index(state[i]) // 3) + abs(i % 3 - goal_state.index(state[i]) % 3)

    # Calculate h(n)
    h = p_n + 3 * s_n

    return h

In [ ]:
# Reading the input file to get the start state and the goal state of the 8-puzzle problem
import numpy as np

# Read the file
with open('states.txt', 'r') as f:
    start_state = []
    goal_state = []
    for line in f:
        if line.strip() == "":  # If we encounter a blank line, we switch to reading the goal state
            break
        row = []
        for num in line.split():
            if num == '*':
                row.append(0)
            else:
                row.append(int(num))
        start_state.append(row)

    for line in f:  # Continue reading the file for the goal state
        row = []
        for num in line.split():
            if num == '*':
                row.append(0)
            else:
                row.append(int(num))
        goal_state.append(row)

start_state = np.array(start_state)
goal_state = np.array(goal_state)

# Print the start and goal states
print(start_state)
print(goal_state)

In [ ]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent

    def g(self):
        return 0 if self.parent is None else self.parent.g() + 1

    def h(self, goal):
        return np.sum(np.abs(self.state - goal))

    def f(self, goal):
        return self.g() + self.h(goal)

def get_neighbors(state):
    # Find the 0 tile
    i, j = np.where(state == 0)

    # List of possible moves
    moves = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

    # Neighbors list
    neighbors = []

    # For each possible move
    for move in moves:
        # If the move is within the bounds of the puzzle
        if 0 <= move[0] < 3 and 0 <= move[1] < 3:
            # Copy the current state to create a new state
            new_state = np.copy(state)
            # Swap the 0 tile with the neighboring tile
            new_state[i, j], new_state[move] = new_state[move], new_state[i, j]
            # Add the new state to the list of neighbors
            neighbors.append(new_state)

    return neighbors

In [ ]:
# A* implementation using the step-by-step pseudocode from the lecture slides
def a_star(start, goal):
    OPEN = [Node(start)]
    f = OPEN[0].f(goal)

    if len(OPEN) == 0:
        break
    else:
        while open_list:
        # Find the node with the lowest f score in the open list
        current = min(OPEN, key=lambda o:o.f)
        
        # If the current node is the goal, then we begin reconstructing the path
        if np.array_equal(current.state, goal.state):
            path = []
            while current.parent:
                path.append(current)
                current = current.parent
            path.append(current)
            return path[::-1]
        
        # Remove the current node from the open list and add it to the closed list
        open_list.remove(current)
        closed_list.append(current)
        
        # For each neighbor of the current node
        for neighbor in get_neighbors(current):
            if any(np.array_equal(neighbor.state, node.state) for node in closed_list):
                continue
            if any(np.array_equal(neighbor.state, node.state) for node in open_list):
                new_g = current.g + current.move_cost(neighbor)
                existing_node = [node for node in open_list if np.array_equal(node.state, neighbor.state)][0]
                if existing_node.g > new_g:
                    existing_node.g = new_g
                    existing_node.parent = current
            else:
                neighbor.g = current.g + current.move_cost(neighbor)
                neighbor.h = heuristic(neighbor, goal)
                neighbor.parent = current
                open_list.append(neighbor)
    return None

    

In [ ]:
def a_star(start, goal, heuristic):
    # Initialize both open and closed list
    open_list = [start]
    closed_list = []
    
    # While the open list is not empty
    while open_list:
        # Find the node with the lowest f score in the open list
        current = min(open_list, key=lambda o:o.f)
        
        # If the current node is the goal, then we begin reconstructing the path
        if np.array_equal(current.state, goal.state):
            path = []
            while current.parent:
                path.append(current)
                current = current.parent
            path.append(current)
            return path[::-1]
        
        # Remove the current node from the open list and add it to the closed list
        open_list.remove(current)
        closed_list.append(current)
        
        # For each neighbor of the current node
        for neighbor in get_neighbors(current):
            if any(np.array_equal(neighbor.state, node.state) for node in closed_list):
                continue
            if any(np.array_equal(neighbor.state, node.state) for node in open_list):
                new_g = current.g + current.move_cost(neighbor)
                existing_node = [node for node in open_list if np.array_equal(node.state, neighbor.state)][0]
                if existing_node.g > new_g:
                    existing_node.g = new_g
                    existing_node.parent = current
            else:
                neighbor.g = current.g + current.move_cost(neighbor)
                neighbor.h = heuristic(neighbor, goal)
                neighbor.parent = current
                open_list.append(neighbor)
    return None